## A Decorator for Timing a Function

In the Python example *GeneratorContextManager*, the purpose of the context manager was to time the code wrapped by the manager.  In the *Python Cookbook, 3rd ed.*, timing is used as an example for both ContextManager and Decorator recipes.  Following on this, the following example tries to duplicate the earlier functionality but is a means to discover that the earlier GeneratorContextManager does a better job for the specific features there.

The primary structural difference in the timed code is that the contextmanager works on a listing of code inside the *with* block whereas the decorator requires a defined function to be timed.  In the case of decorating a preexisting function, say from an imported library module, then the "@" syntax is replaced by calling the decorator directly.

The following defines a decorator using time.perf_counter twice to calcuate the duration of the wrapped function call.  The wrapper function is additionally passed a keyword argument 'label', used subsequently to identify the results. In order to not pass the label argument to the wrapped function, it is saved and then removed just before the call.  It is not particularly elegant, but the alternatives are discussed below. 

In [1]:
from functools import wraps
from time import perf_counter
from array import array

def dec_time(func):
    '''
    Decorator to time wrapped function
    '''  
    @wraps(func)
    def wrapper(*args, **kwargs):
        t0 = perf_counter()
        label = kwargs['label']
        del kwargs['label']
        result = func(*args, **kwargs)
        t1 = perf_counter()
        return (label, (t1 - t0), result)
    return wrapper  


Below shows how the decorator is used to parallel the contextmanager example by timing two array creation calls (slightly different in the syntax and very different in the timing result) inside the function *total_time* which itself is timed.  In order to accomplish this with the decorator, an additional function which takes the decorator is first created and then used to wrap each of the functional calls.  This enables the ability to wrap a sequence of calls rather than only treating individual calls. 

Returning the label, timing and function result allows for more usability
The timing results are very comparable to those from the other example.

In [2]:
@dec_time
def time_func(func, label=''): 
    func() 
     
def total_time():    
    label, t, _ = time_func(lambda: array('d', [0] * 1000000), label='inner: ') 
    print("{0} - {1:1.5f}".format(label, t))     
    
    label, t, _ = time_func(lambda: array('d', [0]) * 1000000, label='outer: ') 
    print("{0} - {1:1.5f}".format(label, t))     

label, t, _ = time_func(total_time, label='total: ')
print("{0} - {1:1.5f}".format(label, t))


inner:  - 0.05163
outer:  - 0.00130
total:  - 0.05316


Using the decorator itself as the receiver of the label argument does not allow for the per-call assignment.

The addition of the *time_func* function is unfortunate and would suggest having a single function, not a decorator, that handles the timing and label more simply.  Or the GeneratorContextManager version is also much more succinct.

Below shows the case of using the decorator on an imported library function. The decorator is called directly as a function.  The wrapping function is called with both the expected arguments of the wrapped *randint* function and the keyword label argument.   

In [3]:
from random import randint
timed_rand_int = dec_time(randint)
label, t, r = timed_rand_int(0, 100, label='timed randint: ')
print("{0} - {1:1.5f} - result: {2}".format(label, t, r))


timed randint:  - 0.00002 - result: 40
